### Michał Ilski 250079
### Lista 4

### Zadanie 1.

W zadaniu pierwszym należy napisać funkcję ilorazyRoznicowe, która służy do obliczania ilorazów różnicowych dla danych par $(x_i, f(x_i)), i \in \{0,...,n-1\}$. Dzięki obliczonym ilorazom jesteśmy w stanie napisać wzór na wielomian interpolacyjny $p(x)$ w postaci Newtona:
<center>$p(x) = \sum_{k=0}^{n}f[x_0,x_1,...,x_k]\prod_{j=0}^{k-1}(x-x_j)$</center>  
gdzie $f[x_0,x_1,...,x_k]$ stanowią obliczone ilorazy różnicowe, a $x_j$ dane węzły.

Funkcja na wejściu otrzymuje wektor $[x_0, x_1, ... , x_n]$ stanowiący węzły, oraz wektor wartości funckcji $f$ w odpowiednich punktach $[f(x_0), f(x_1), ... , f(x_n)]$.

In [ ]:
function ilorazyRoznicowe(x::Vector{Float64}, f::Vector{Float64})
    fx = Vector{Float64}(undef,length(x))
    cp = copy(f)
    
    for n in 1:length(x)
        fx[n] = cp[1]
        
        for i in 1:length(x)-n
            cp[i] = (cp[i+1] - cp[i]) / (x[i+n] - x[i])
        end
    end
    return fx
end

Opis kodu:  
Na początku inicjuję wektor (zgodnie z treścią zadania nie korzystam z tablicy dwuwymiarowej) fx, który będzie zawierał obliczone wartości ilorazów różnicowych. Zmienna cp to kopia wektora wartości funkcji. Będzie stale nadpisywać w nim wartości, w kolejnych iteracjach pętli w celu zaoszczędzenie pamięci.  
Pętla zewnętrzna odpowiada za iterację po kolumnach trójkątnej tablicy, w której są wyliczane ilorazy różnicowe (wizualizacja tablicy na slajdzie 16, wykład 6). Do wektora fx dodajemy najwyżej umieszczoną wartość z kolumny, czyli cp[1] = $f[x_0,...,x_k]$ dla k-tej iteracji (kolumny). Wartości w każdej następnej kolumnie obliczam według wzoru:
<center>$cp[i] := \frac{(cp[i+1] - cp[i])}{(x[i+n] - x[i])}$</center>  
Który zgodnie z notacją z wykładu ma postać:
<center>$f[x_i,...x_{k+i}] := \frac{f[x_{i+1},...,x_{k+i}] - f[x_i,...,x_{k+i-1}]}{x_{i+k} - x_i}$</center>
Na końcu zwracany jest wektor fx, zawierający obliczone ilorazy różnicowe $[f[x_0], ..., f[x_0,...,x_n]]$.

Poniżej test dla wartości z zadania 4. z listy 4. z ćwiczeń:

In [ ]:
x = Vector([-2.0,-1.0,0.0,1.0,2.0,3.0])
f = Vector([-25.0,3.0,1.0,-1.0,27.0,235.0])

In [ ]:
fx = ilorazyRoznicowe(x,f)

Czyli obliczony wielomian ma postać $p(x) = -25 + 28(x+2) - 15(x+2)(x+1) + 5(x+2)(x+1)x + (x+2)(x+1)x(x-1)(x-2)$.

### Zadanie 2.

Teraz mając już obliczone ilorazy różnicowe, należy zaimplementować funkcję warNewton, obliczającą wartość tego wielomianu w punkcie $t$, w złożoności $\mathcal{0}(n)$, gdzie $n$ stanowi stopień wielomianu. Wykorzystany został do tego uogólniony algorytm Hornera. Zgodnie z algorytmem, w n-k-tej iteracji obliczamy:
<center>$w_k(x) := f[x_0,...,x_k] + (x-x_k)w_{k+1}(x)$</center>
zaczynając od $w_n$ i iterując do $w_0$.

In [ ]:
function warNewton(x::Vector{Float64}, fx::Vector{Float64}, t::Float64)
    res = fx[length(x)]
    for i in 1:length(x)-1
        n = length(x)-i
        res = fx[n] + (t-x[n])*res
    end
    return res
end

Opis kodu:  
Funkcja na wejściu przyjmuje niezbędne wartości, aby móc skorzystać z ww. wzoru, tzn. wektor ilorazów różnicowych oraz węzłów i argument $t$, dla którego wyliczamy wartość wielomianu. Na początku inicjuję zmienną res wartością $f[x_0,...,x_n]$. Następnie w pętli obliczam kolejne $w_k$, aż zejdę do $w_0$. Z racji, że potrzebuję do obliczenia $w_k(t)$ znać jedynie wartość $w_{k+1}(t)$ (oraz oczywiście $f[x_0,...,x_k]$), to nie przechowuję zbędnych informacji o starszych $w_{k+i}, i \in {2,...,n}$. Po wykonaniu się pętli w zmiennej res mam wartość $w_0(t)$, którą zwracam.  
W pętli wykonują się obliczenia o stałej złożoności $\mathcal{O}(1)$, a sama pętla wykona się się n-1 razy, stąd ostatecznie złożonść funkcji wynosi $\mathcal{O}(n)$.

Dla poprzedniego przykładu wiemy, że $f(3) = 235$. Z racji, że $x=3$ jest węzłem, powinno zajść również $p(3) = 235$.

In [ ]:
t = 3.0
warNewton(x,fx,t)

### Zadanie 3.

W tym zadaniu podobnie skorzystam z obliczonych wartości ilorazów różnicowych oraz węzłów, które zostaną przekazane jako wartości wejściowe w postaci wektorów. Celem zadania jest jest implementacji funkcji "naturalna", która oblicza w złożności $\mathcal{O}(n^2)$ współczynniki postaci naturalnej $a_0, ..., a_n$. Wykorzystany algorytm bazuje na algorytmie z zadania wyżej (czyli Hornera). Modyfikacja polega na tym, że z każdą iteracją pętli zewnętrznej, wewnętrzna aktualizuje obliczone do tej pory $w_k(x)$.

In [ ]:
function naturalna(x::Vector{Float64}, fx::Vector{Float64})
    n = length(x)-1
    res = Vector{Float64}(undef, n+1)
    res[n+1] = fx[n+1]
    for i in n:-1:1
        res[i] = fx[i] - res[i+1]*x[i]
        for k in i+1:n
            res[k] = res[k] - res[k+1]*x[i]
        end
    end
    return res
end

Opis kodu:  
W pierwszej kolejności inicjuję zmienną $n$ długością tablicy węzłów - 1, aby nie obliczać jej wielokrotnie. Następnie rezerwuję w pamięci miejsce na n+1 elementowy wektor $[a_0,...,a_n]$. Podstawiam za $a_n := f[x_0,...,x_n]$. Następnie w pętli obliczam $w_i(x)$, korzystając z algorytmu Hornera. W wewnętrznej pętli dodatkowo aktualizuję dotychczas policzone $w_{i+1},...,w_{n}$ uwzględniając nowy czynnik $x_i$. Na końcu zwracam wektor $[a_0,...a_n]$. Sam algorytm wykonuje w pętli zewnętrznej obliczenia w stałej złożności, oraz zawiera pętlę wewnętrzną, która również wykonuje obliczenia w stałej złożoności czasowej. Liczbę wykonań obu pętli można ograniczyć z góry przez n, przez co złożność całego algorytmu wynosi $O(n^2)$.

W zadaniu 1. dostaliśmy wielomian w postaci:
<center>$p(x) = -25 + 28(x+2) - 15(x+2)(x+1) + 5(x+2)(x+1)x + (x+2)(x+1)x(x-1)(x-2)$</center>
Można go uprościć ręcznie do postaci:
<center>$p(x) = x^5 - 3 x + 1$</center>
Oznacza to, że dla tego wielomianu wektor $[a_0,...,a_5]$ ma postać $[1,-3,0,0,0,1]$.

In [ ]:
x = Vector([-2.0,-1.0,0.0,1.0,2.0,3.0])
fx = Vector([-25.0,28.0,-15.0,5.0,0.0,1.0])
naturalna(x, fx)

Jak widać zwracany wektor zgadza się z tym obliczonym ręcznie.

### Zadanie 4.

W zadaniu została zaimplementowana funkcja interpolująca zadaną funkcję $f(x)$ na przedziale $[a,b]$ przy pomocy wielomianu $p(x)$ stopnia $n$ w postaci Newtona. Funkcja ma zadanie wygenerować ilorazy różnicowe (dla $n+1$ równoodległych węzłów z $[a,b]$) przy pomocy funkcji z zadania 1, następnie obliczyć wartości wielomianu interpolacyjnego na przedziale $[a,b]$, korzystając z funkcji z zadania 2 (w naszym przypadku jest to 100 wartości $p(x)$ dla równoodległych $x \in [a,b]$). Na końcu zostaną przedstawione wykresy $f(x)$ oraz $p(x)$ ($f(x)$ liczona w tych samych punktach co $p(x)$).

Importowanie Plots do wykresów:

In [ ]:
using PyPlot

Pomocniczna funkcja linspace, generująca $n+1$ równoodległych wartości z przedziału $[a,b]$:


In [ ]:
function linspace(a::Float64, b::Float64, n::Int)
    h = (b-a)/n
    return [a+k*h for k in 0:n]
end

Implementacja funkcji z zadania rysujNNfx:

In [ ]:
function rysujNnfx(f,a::Float64,b::Float64,n::Int)
    x = linspace(a,b,n)
    f_values = [f(xi) for xi in x]
    fx = ilorazyRoznicowe(x, f_values)
    t_s = linspace(a,b,100)
    p_values = [warNewton(x,fx,t) for t in t_s]
    f_values = [f(t) for t in t_s]
    plot(t_s, f_values, t_s, p_values)
    legend(["f(x)", "p(x)"])
end

Opis kodu:  
Na początku wyznaczam wektor węzłów $x$ korzystając z mojej funkcji pomocniczej. Następnie wyznaczam wektor f_values = $[f(x_0),...,f(x_n)]$. Później kolejno obliczam ilorazy różnicowe dla $f(x)$, wyznaczam wektor 100  równoodległych $x$ z przedziału $[a,b]$ i obliczam dla nich wartości $p(x)$ i $f(x)$. Na końcu generuję wykresy z legendą.

In [ ]:
rysujNnfx(sin, -4.0, 4.0, 4)

Na powyższym testowym wykresie widzimy funkcję $sin(x)$ oraz wielomian $p(x)$ na przedziale $[-4,4]$. Wielomian został wygenerowany dla 5 węzłów $[-4,-2,0,2,4]$. Łatwo można zauważyć, że faktycznie wykresy przecinają się dla tych $x$.

### Zadanie 5.

W zadaniu zostały wykonane testy na dwóch różnych funkcjach dla 5, 10 i 15 węzłów.

Pierwsza funkcja to $g(x) = e^x$ na przedziale $[0,1]$:

In [ ]:
g(x::Float64) = exp(x)

Kolejno wykresy dla 5, 10 i 15 węzłów:

In [ ]:
rysujNnfx(g, 0.0, 1.0, 5)

In [ ]:
rysujNnfx(g, 0.0, 1.0, 10)

In [ ]:
rysujNnfx(g, 0.0, 1.0, 15)

Druga funkcja to $x^2 sin(x)$ na prziedziale $[-1,1]$:

In [ ]:
g(x::Float64) = x^2 * sin(x)

Kolejno wykresy dla 5, 10 i 15 węzłów:

In [ ]:
rysujNnfx(g, -1.0, 1.0, 5)

In [ ]:
rysujNnfx(g, -1.0, 1.0, 10)

In [ ]:
rysujNnfx(g, -1.0, 1.0, 15)

Dla każdego z powyższych wykresów nie jesteśmy w stanie odróżnić $f(x)$ od $p(x)$. Oznacza to, że wygenerowane wielomiany bardzo dobrze przybliżają funkcję $f(x)$ nawet dla 5 węzłów. Zminimalizowanie błędu pomiędzy $f(x)$ i $p(x)$ jest w dużej mierze zasługą użycia węzłów równoodległych.

### Zadanie 6.

W tym zadaniu ponownie przetestujemy dotychczas napisane funkcje, jednak na innych (trudniejszych do interpolacji) danych.

Pierwsza funkcja to $g(x)=|x|$ na przedziale $[-1,1]$.

In [ ]:
 g(x::Float64) = abs(x)

Kolejno wykresy dla 5, 10 i 15 węzłów:

In [ ]:
rysujNnfx(g,-1.0,1.0,5)

In [ ]:
rysujNnfx(g,-1.0,1.0,10)

In [ ]:
rysujNnfx(g,-1.0,1.0,15)

W tym przypadku widzimy, że przybliżenie nie jest już tak dokładne jak w zadaniu 5. Co więcej większa liczba węzłów wcale nie poprawia rezulatatów. Problemem funkcji $g(x)=|x|$ jest fakt, że nie jest ona różniczkowalna na przedziale $[-1,1]$. Alternatywnym rozwiązaniem może być interpolacja osobno funkcji $g_1(x)=-x$ dla $x<0$ oraz $g_2(x)=x$ dla $x>0$.

In [ ]:
g(x::Float64) = 1/(1+x^2)

In [ ]:
rysujNnfx(g,-5.0,5.0,5)

In [ ]:
rysujNnfx(g,-5.0,5.0,10)

In [ ]:
rysujNnfx(g,-5.0,5.0,15)

W tym przypadku również istnieje widoczna rozbieżność między $f(x)$ a $p(x)$. Co więcej dla większych stopni wielomianu, błędy rosną. Występuje tu tzw. zjawisko Runge'ego, charakterystyczne dla interpolacji przy równoodległych węzłach i wysokich stopniach wielomianów. Widzimy, że dla 15 węzłów wielomian w środkowej części prawie pokrywa się z $f(x)$, natomiast na krańcach występują znaczące błędy. Rozwiązaniem tego problemu jest zagęszczenie węzłów na krańcach. Takie węzły mogą być miejscami zerowymi wielomianu Czebyszewa n-tego stopnia.